# Installations

In [ ]:
!pip install fastapi
!pip install fastapi uvicorn
!pip install colabcode
!pip install "uvicorn[standard]"
!pip install python-multipart
!ngrok config add-authtoken 2fJ2rFEdJMTj8I6ngtjEdLhl7MT_SqFfoJr16MPbKcCvukkn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 946.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 905.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import Libraries

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
# import uvicorn
from typing import Union
from fastapi import FastAPI
from colabcode import ColabCode
from google.colab import drive
from fastapi import UploadFile, File
import shutil
from google.colab import drive
from pyngrok import ngrok

# Global Objects

In [ ]:
cc = ColabCode(port=8000, code=False)
app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# Constants

In [ ]:
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/3DCNN-model'
video_path = '/content/drive/MyDrive/RWF-2000/val/Fight/zOqs7Oh9oDM_2.avi'

Mounted at /content/drive


In [ ]:
def load_and_read_video_frames(path, num_frames, frame_height, frame_width,num_channels):
    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(num_frames):
        ret,frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        normalized_frame = frame / 255.0
        frames.append(normalized_frame)
    cap.release()
    frames = np.array(frames)
    return frames

# App Root

In [ ]:
# @app.get("/")
# def read_root():
#     return {"HELLOO":"WORLD"}

# App Startup

In [ ]:
@app.on_event("startup")
def load_model():
    global model
    model = tf.keras.models.load_model(model_path)
    model.summary()

<ipython-input-6-6e42f0b48b0e>:1: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


# Post Prediction

In [ ]:
@app.post("/")
async def get_predictions(video: UploadFile = File(...)):
    try:
      dest_path = f"/content/drive/MyDrive/{video.filename}"
      with open(dest_path, "wb") as f:
       shutil.copyfileobj(video.file, f)
      video_data = load_and_read_video_frames(dest_path,150,160,160,3)
      video_data = np.expand_dims(video_data, axis=0)
      prediction = model.predict(video_data)
      if(prediction.round() == 1.0):
        predicted_class = "Violent"
      else:
        predicted_class = "Nonviolent"
      return {"Prediction": predicted_class}

    except Exception as e:
         return {"prediction": "error", "exception":str(e)}

# Run App

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


# cc.run_app(app=app)

INFO:     Started server process [138]
INFO:uvicorn.error:Started server process [138]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.


Public URL: https://2e4e-35-245-59-75.ngrok-free.app
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 160, 160,    0         
                             3)]                                 
                                                                 
 conv3d (Conv3D)             (None, 150, 160, 160, 1   448       
                             6)                                  
                                                                 
 conv3d_1 (Conv3D)           (None, 150, 160, 160, 1   784       
                             6)                                  
                                                                 
 batch_normalization (Batch  (None, 150, 160, 160, 1   64        
 Normalization)              6)                                  
                                                                 
 max_poo

INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1/1 [==============================] - 3s 3s/step
INFO:     154.187.166.231:0 - "POST / HTTP/1.1" 200 OK
INFO:     154.187.166.231:0 - "POST / HTTP/1.1" 200 OK
INFO:     154.187.166.231:0 - "POST / HTTP/1.1" 200 OK
INFO:     154.187.166.231:0 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [138]
INFO:uvicorn.error:Finished server process [138]
